# Дебажим w2v процессор

In [66]:
import gensim 
print(gensim.__version__)

import numpy as np
from tqdm import tqdm

import csv

3.8.0


## 1. Распаковка эмбедингов

In [2]:
MODEL_PATH = "./model.bin"
w2v_model = gensim.models.KeyedVectors.load_word2vec_format(MODEL_PATH, binary=True)

In [8]:
w2v_model.wv.get_vector("путин_NOUN")[:5]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


array([ 0.34272522, -0.08931037, -0.449796  , -0.48029754,  0.2616737 ],
      dtype=float32)

In [20]:
d = w2v_model.vector_size
V = len(w2v_model.vocab) + 2

# "#UNK" для неизвестных слов
# "#PAD" заглушки, чтобы были одинаковые размерности у матриц

# Мама мыла раму 
# Филипп закомитил код на гитхаб  

# Мама мыла раму  #PAD  #PAD
# #UNK закомитил код на #UNK

word2index = {"#UNK": 0, "#PAD":1}

embedding = np.zeros((V, d))

for i,word in tqdm(enumerate(w2v_model.vocab.keys())):
    word2index[word] = i + 2
    embedding[i+2] = w2v_model.wv.get_vector(word)
    
assert embedding.shape == (V, d)
assert len(word2index) == V

0it [00:00, ?it/s]/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
249318it [00:03, 64659.96it/s]


In [29]:
np.save("embedding.npy", embedding)

In [31]:
import pickle 

with open("word2index.pickle", "wb") as f:
    pickle.dump(word2index, f)

## 2. Предобработка текста

In [184]:
import sys
from pymystem3 import Mystem

# Таблица преобразования частеречных тэгов Mystem в тэги UPoS:
mystem2upos = {'A': 'ADJ', 'ADV': 'ADV', 'ADVPRO': 'ADV', 'ANUM': 'ADJ',
             'APRO': 'DET', 'COM': 'ADJ', 'CONJ': 'SCONJ', 'INTJ': 'INTJ',
             'NONLEX': 'X', 'NUM': 'NUM', 'PART': 'PART', 'PR': 'ADP', 
             'S': 'NOUN', 'SPRO': 'PRON', 'UNKN': 'X', 'V': 'VERB'}

m = Mystem(entire_input=False)

fuck_cnt = 0
all_cnt = 0

def tag_mystem(text='Текст нужно передать функции в виде строки!', mapping=None, postags=True):
    # если частеречные тэги не нужны (например, их нет в модели), выставьте postags=False
    # в этом случае на выход будут поданы только леммы
    
    global fuck_cnt
    global all_cnt

    processed = m.analyze(text)
    tagged = []
    
    for w in processed:
        all_cnt += 1
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if mapping:
                if pos in mapping:
                    pos = mapping[pos]  # здесь мы конвертируем тэги
                else:
                    pos = 'X'  # на случай, если попадется тэг, которого нет в маппинге
            tagged.append(lemma.lower() + '_' + pos)
        except:
            fuck_cnt += 1
            continue  # игнорируем все ошибки 
    if not postags:
        tagged = [t.split('_')[0] for t in tagged]
    return " ".join(tagged)

In [86]:
# Пример
texts = ["Мама мыла раму", "desc Филипп закомитил коды на гитхаб"] 

tag_mystem(texts[1], mapping=mystem2upos)

'филипп_NOUN закомитила_NOUN код_NOUN на_ADP гитхаб_NOUN'

In [87]:
tag_mystem(texts[1], mapping=mystem2upos, postags=False)

'филипп закомитила код на гитхаб'

Основной цикл предобработки.

In [185]:
!rm 'cian_data_lemm_v1.csv'

In [186]:
!touch 'cian_data_lemm_v1.csv'

Код считывает строчку, предобрабатывает её и записывает. 

In [ ]:
f1 = open('cian_data_v1.txt', "r")
f2 = open('cian_data_lemm_v1.csv', "w")

reader = csv.DictReader(f1, delimiter="\t")
row1 = next(reader).copy()
fieldnames = list(row1.keys()) + ['описание тэги', 'описание без тэгов']

writer = csv.DictWriter(f2, fieldnames=fieldnames, delimiter="\t")

f1.close()
f1 = open('cian_data_v1.txt', "r")
reader = csv.DictReader(f1, delimiter="\t")

# НАДО ДОПИСАТЬ В ТАБЛИЦУ НАЗВАНИЯ СТОЛБЦОВ!

cnt = 0
for row in tqdm(reader):
    cnt += 1
    row['описание тэги'] = tag_mystem(row['описание'], mapping=mystem2upos)
    row['описание без тэгов'] = tag_mystem(row['описание'], mapping=mystem2upos, postags=False)
    writer.writerow(row)
    # f2.write(text + "\n")
    
#     if cnt > 20:
#         break 
        
f1.close()
f2.close()

##  3. Векторайзер

In [32]:
import pandas as pd
import numpy as np

df = pd.read_csv('cian_data_lemm_v1.csv', sep="\t", header=None)

y = df[37].apply(np.log).to_numpy()
texts = df[38].to_numpy()

In [29]:
import pickle 

with open("word2index.pickle", "rb") as f:
    word2index = pickle.load(f)
    
embedding = np.load("embedding.npy")

In [52]:
class W2vVecorizer:
    
    def __init__(self, word2index, embedding):
        self.word2index = word2index
        self.embedding = embedding
        self.vocabulary = set()
        
        self.nice_cnt = 0
        self.all_cnt = 0 
    
    def fit(self, X):
        for text in X:
            tokens = text.split(" ")
            for tok in tokens:
                self.vocabulary.add(tok)
    
    # без PADDING
    def transform(self, X):
        X_vect = np.zeros((len(X), self.embedding.shape[1]))

        for i, text in enumerate(X):
            vect = np.zeros(self.embedding.shape[1]) # копим вектор 
            cnt = 0
            
            tokens = text.split(" ")
            for tok in tokens:
                self.all_cnt += 1
                if (tok in self.vocabulary) and (tok in self.word2index):
                    vect += self.embedding[self.word2index[tok]]
                    cnt += 1
                    self.nice_cnt += 1
            X_vect[i] = vect/cnt
        return X_vect
    
    def fit_transform(self, X):
        self.fit()
        return self.transform()
    

In [53]:
vectorzer = W2vVecorizer(word2index, embedding)
vectorzer.fit(texts)

In [54]:
X_vect = vectorzer.transform(texts)
X_vect.shape

(2456, 300)

In [56]:
vectorzer.nice_cnt/vectorzer.all_cnt

0.7499925038444569

__Добавить в класс аргументы:__

* Попробовать обучить линейную модель на w2v
* Надо попробовать в наш класс добавить PADD, UNKN 
* Надо поробовать впихнуть tf-idf 
* Сварить ещё одни w2v, но на основе другой модели https://rusvectores.org/ru/models/
* Обучить свой собственный w2v только на описаниях 

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(min_df=2, max_df=0.8)

texts_v = vect.fit_transform(texts)

In [67]:
feature_names = vect.get_feature_names()

tfidf_dict = dict(zip(feature_names, np.array(texts_v.mean(axis=0))[0]))

In [69]:
tfidf_dict['абсолютно_adv']

0.0010757079971086944